In [1]:
!pip install pandas
!pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import pandasql as ps

csv_name = 'cars.csv'

cars = pd.DataFrame(
    {
        'speed': [300, 200, 140],
        'engine': [5, 0, 3],
        'doors': [2, 4, 5],
    }, index=pd.Index(['lambo', 'tesla', 'lr'], name='car')
)

cars

,speed,engine,doors
car,,,
lambo,300,5,2
tesla,200,0,4
lr,140,3,5


In [2]:
cars.to_csv(csv_name)

In [3]:
cars = pd.read_csv(csv_name, index_col='car')
cars_wo_index = pd.read_csv(csv_name)

In [4]:
cars
cars_wo_index

,speed,engine,doors
car,,,
lambo,300,5,2
tesla,200,0,4
lr,140,3,5


,car,speed,engine,doors
0,lambo,300,5,2
1,tesla,200,0,4
2,lr,140,3,5


In [5]:
ti = pd.read_csv('titanic.csv', index_col=0)
ti

,Name,PClass,Age,Sex,Survived,SexCode
PassengerID,,,,,,
1,"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1
2,"Allison, Miss Helen Loraine",1st,2.00,female,0,1
3,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0
4,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1
5,"Allison, Master Hudson Trevor",1st,0.92,male,1,0
...,...,...,...,...,...,...
1309,"Zakarian, Mr Artun",3rd,27.00,male,0,0
1310,"Zakarian, Mr Maprieder",3rd,26.00,male,0,0
1311,"Zenni, Mr Philip",3rd,22.00,male,0,0


In [ ]:
sql = """
select 
       --sex, survived,
       count(Name), count(Age), count(SexCode), count(*)
  from ti
--  group by sex, survived
--  order by sex, survived
"""
sql1 = """
select case when age is null then 'null' else 'not null' end as age_is_null, count(*)
  from ti
  group by case when age is null then 'null' else 'not null' end
"""

In [ ]:
ps.sqldf(sql)
ps.sqldf(sql1)
ps.sqldf("select * from ti where age is null limit 5")
ps.sqldf("select * from ti where age is not null limit 5")

,count(Name),count(Age),count(SexCode),count(*)
0,1313,756,1313,1313


,age_is_null,count(*)
0,not null,756
1,null,557


,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,13,"Aubert, Mrs Leontine Pauline",1st,None,female,1,1
1,14,"Barkworth, Mr Algernon H",1st,None,male,1,0
2,15,"Baumann, Mr John D",1st,None,male,0,0
3,30,"Borebank, Mr John James",1st,None,male,0,0
4,33,"Bradley, Mr George",1st,None,male,1,0


,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,1,"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1
1,2,"Allison, Miss Helen Loraine",1st,2.00,female,0,1
2,3,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0
3,4,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1
4,5,"Allison, Master Hudson Trevor",1st,0.92,male,1,0


In [ ]:
ps.sqldf("select * from ti where PClass='*'")

,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,457,Jacobsohn Mr Samuel,*,None,male,0,0


In [18]:
ti.loc[ti.PClass=='*']

,Name,PClass,Age,Sex,Survived,SexCode
PassengerID,,,,,,
457,Jacobsohn Mr Samuel,*,NaN,male,0,0


In [13]:
ti.PClass=='*'

PassengerID
1       False
2       False
3       False
4       False
5       False
        ...  
1309    False
1310    False
1311    False
1312    False
1313    False
Name: PClass, Length: 1313, dtype: bool

In [152]:
sql_surv_class = """
select 
       PClass, survived, count(*)
  from ti
  group by PClass, survived
  order by 1, 2
"""

In [153]:
ps.sqldf(sql_surv_class)

,PClass,Survived,count(*)
0,*,0,1
1,1st,0,129
2,1st,1,193
3,2nd,0,160
4,2nd,1,119
5,3rd,0,573
6,3rd,1,138


In [154]:
ti.pivot_table(
    index=['Sex'],
    columns=['PClass'],
    values='Age',
    aggfunc='median',
)

PClass,1st,2nd,3rd
Sex,,,
female,38.0,28.0,22.0
male,42.0,28.0,25.5


In [155]:
ti.pivot_table(
    index=['Survived'],
    columns=['PClass'],
    values='Age',
    aggfunc='median',
)

PClass,1st,2nd,3rd
Survived,,,
0,46.0,29.5,25.0
1,36.0,25.0,22.0


In [169]:
data = pd.read_csv('weights.csv', index_col=0)
data.index.name = 'date'
data

,AT0000A18XM4 SW,BE0974268972 BB,US0527691069 US,DE0007164600 GR,US6092071058 US
date,,,,,
2014-01-14,0.1846596556,0.2757063529,0.3034246891,0.0428412772,0.1933680252
2014-01-15,0.2258753717,0.1478614681,0.1512114493,0.2240741840,0.2509775269
2014-01-16,0.2610167988,0.3342126804,0.2933360672,0.0157443424,0.0956901113
2014-01-17,0.1601680600,0.2310344514,0.0926736430,0.2690502767,0.2470735689
2014-01-18,0.0571617706,0.2541460418,0.0272860256,0.4806599417,0.1807462203
...,...,...,...,...,...
2018-02-28,0.1238206028,0.1678991025,0.1208435436,0.0887801982,0.4986565529
2018-03-01,0.0350426536,0.2896235744,0.2208713751,0.1363664796,0.3180959173
2018-03-02,0.3971553478,0.3620252230,0.0505537488,0.0202470952,0.1700185852


In [230]:
sql_data = """
select case
         when sum_of_columns = 1 then '==1'
         when sum_of_columns < 1 then '<1'
         when sum_of_columns > 1 then '>1'
         else '?'
       end as sum_rel_to_1,
       sum_of_columns, count(*) as qnty, min(date) as min_date, max(date) as max_date
  from (
    select date, "AT0000A18XM4 SW" + "BE0974268972 BB" + "US0527691069 US" + "DE0007164600 GR" + "US6092071058 US" as sum_of_columns
      from data
      --where sum_of_columns=1
  ) a
  group by sum_of_columns
  order by qnty desc
"""

# pd.set_option("display.precision", 40)
data_cols_sum = ps.sqldf(sql_data)
data_cols_sum.to_csv('weights_cols_summary.csv')
with pd.option_context('display.precision', 20, 'display.max_rows', 10):
    data_cols_sum

,sum_rel_to_1,sum_of_columns,qnty,min_date,max_date
0,==1,1.00000000000000000000,165,2014-01-15,2018-02-13
1,>1,1.00000000000009992007,46,2014-01-14,2018-01-04
2,>1,1.00000000000029998226,45,2014-01-21,2018-02-02
3,<1,0.99999999999979993781,44,2014-03-15,2018-01-02
4,>1,1.00000000000020006219,37,2014-02-19,2018-01-27
...,...,...,...,...,...
145,<1,0.99999999999900024417,1,2016-04-12,2016-04-12
146,<1,0.99999999999894995106,1,2017-11-23,2017-11-23
147,<1,0.99999999999892008606,1,2015-12-31,2015-12-31
148,<1,0.99999999999869992884,1,2017-02-23,2017-02-23


In [273]:
import numpy as np


def quantile_high(x):
    return x.quantile(.99, interpolation='nearest')


def quantile_low(x):
    return x.quantile(.01, interpolation='nearest')


def quantile_50(x):
    return x.quantile(.5, interpolation='nearest')


def second(x):
    return sorted(x.array)[1]


def before_last(x):
    return sorted(x.array)[-2]


with pd.option_context('display.precision', 20, 'display.max_rows', 20):
    data.agg([np.min, second, quantile_low, np.max, before_last, quantile_high, np.sum, np.median, quantile_50])

,AT0000A18XM4 SW,BE0974268972 BB,US0527691069 US,DE0007164600 GR,US6092071058 US
amin,0.00006324065623360000,0.00001772070035710000,0.00033072555740600002,0.00000028957406732000,0.00013168698548799999
second,0.00056832608789499995,0.00005853430333830000,0.00060501499727000005,0.00089959071450700003,0.00085935616687600003
quantile_low,0.00350278153427000015,0.00493473843680999977,0.00415517367883999969,0.00620237987144000007,0.00723891980141999972
amax,0.58878311877500000282,0.63858269868700001659,0.79298245820900004066,0.85554993405599999434,0.62190366982900002757
before_last,0.51422848409499999356,0.63294675020099999685,0.71303513476600000054,0.58111496260200001807,0.55771674779000002253
quantile_high,0.45644152804600002238,0.50502054067799995174,0.50583069728300000811,0.47684946836899999578,0.48791753556700001582
sum,207.61007571537618332513,219.56080655585805061492,221.51103382852187451135,217.21963889178613271724,215.09844500844010894980
median,0.18887609421900000695,0.20149625733699999541,0.20061813022700000775,0.19873642680199998800,0.20404436304299999883
quantile_50,0.18887609421900000695,0.20149625733699999541,0.20061813022700000775,0.19873642680199998800,0.20404436304299999883


In [264]:
s = pd.Series([2,5,1,4,3])
s.sort_values()
sorted(s.array)[-2]

2    1
0    2
4    3
3    4
1    5
dtype: int64

4

In [19]:
fines = pd.read_json('fines.jcon')
pd

ValueError: Unexpected character found when decoding 'false'